### import required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import warnings
warnings.filterwarnings('ignore')
from scipy.sparse import hstack

### importing datasets

In [2]:
train = pd.read_csv('Final_Train_Dataset.csv')
test = pd.read_csv('Final_Test_Dataset.csv')

In [3]:
train.head()

,Unnamed: 0,experience,job_description,job_desig,job_type,key_skills,location,salary,company_name_encoded
0,0,5-7 yrs,Exp: Minimum 5 years;Good understanding of IOC...,Senior Exploit and Vulnerability Researcher,NaN,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri),6to10,3687
1,1,10-17 yrs,He should have handled a team of atleast 5-6 d...,Head SCM,NaN,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458
2,2,5-9 yrs,Must be an effective communicator (written & s...,Deputy Manager - Talent Management & Leadershi...,Analytics,"HR Analytics, Employee Engagement, Training, S...",Delhi NCR,15to25,4195
3,3,7-10 yrs,7 - 10 years of overall experience in data e...,Associate Manager Data Engineering,Analytics,"SQL, Javascript, Automation, Python, Ruby, Ana...",Bengaluru,10to15,313
4,4,1-3 yrs,Chartered Accountancy degree or MBA in Finance...,TS- GSA- Senior Analyst,NaN,"accounting, finance, cash flow, financial plan...",Gurgaon,3to6,1305


### preprocessing the text columns

In [4]:
train = train.dropna(subset=['key_skills'])

df_train = train[['key_skills', 'job_desig', 'job_description', 'location', 'job_type', 'experience','salary']]
df_test = test[['key_skills', 'job_desig', 'job_description', 'job_type', 'experience', 'location']]

In [5]:
def cleaning_skills(skl):
    skills = str(skl).lower()
    skills = re.sub(r'\s+', ' ', skills)
    skills = re.sub(r',','', skills)
    skills = re.sub(r'\...','', skills)
    return skills

df_train['skills_cleaned'] = df_train['key_skills'].apply(cleaning_skills)
df_test['skills_cleaned'] = df_test['key_skills'].apply(cleaning_skills)

In [6]:
def clean_job_designation(desig):
    job_desig = desig.lower()
    job_desig = re.sub(r'[^a-z]', ' ', job_desig)
    job_desig = re.sub(r'\s+', ' ', job_desig)
    return job_desig

df_train['desig_cleaned'] = df_train['job_desig'].apply(clean_job_designation)
df_test['desig_cleaned'] = df_test['job_desig'].apply(clean_job_designation)

In [7]:
train['job_description'].fillna('missing', inplace=True)
test['job_description'].fillna('missing', inplace=True)

def clean_job_description(job):
    job_desc = str(job).lower()
    job_desc = re.sub(r'[^a-z]', ' ', job_desc)
    job_desc = re.sub(r'\s+', ' ', job_desc)
    return job_desc

df_train['job_desc_cleaned'] = df_train['job_description'].apply(clean_job_description)
df_test['job_desc_cleaned'] = df_test['job_description'].apply(clean_job_description)

In [8]:
def clean_location(loc):
    location = loc.lower()
    location = re.sub(r'[^a-z]', ' ', location)
    location = re.sub(r'\s+', ' ', location)
    return location

df_train['loc_cleaned'] = df_train['location'].apply(clean_location)
df_test['loc_cleaned'] = df_test['location'].apply(clean_location)

In [9]:
train['job_type'].fillna('missingjobtype', inplace=True)
train['job_type'].replace('Analytics', 'analytics', inplace=True)
train['job_type'].replace('Analytic', 'analytics', inplace=True)
train['job_type'].replace('ANALYTICS', 'analytics', inplace=True)
train['job_type'].replace('analytic', 'analytics', inplace=True)

test['job_type'].fillna('missingjobtype', inplace=True)
test['job_type'].replace('Analytics', 'analytics', inplace=True)
test['job_type'].replace('Analytic', 'analytics', inplace=True)
test['job_type'].replace('ANALYTICS', 'analytics', inplace=True)
test['job_type'].replace('analytic', 'analytics', inplace=True)

df_train['job_type_cleaned'] = train['job_type'] 
df_test['job_type_cleaned'] = test['job_type']

In [10]:
def min_exp(exp):
    val = re.sub(r'-',' ', exp)
    val = val.split(' ')
    val = int(val[0])
    return val

def max_exp(exp):
    val = re.sub(r'-',' ', exp)
    val = val.split(' ')
    val = int(val[1])
    return val

df_train['min_exp'] = df_train['experience'].apply(min_exp)
df_train['max_exp'] = df_train['experience'].apply(max_exp)

df_test['min_exp'] = df_test['experience'].apply(min_exp)
df_test['max_exp'] = df_test['experience'].apply(max_exp)

In [11]:
df_train.head()

,key_skills,job_desig,job_description,location,job_type,experience,salary,skills_cleaned,desig_cleaned,job_desc_cleaned,loc_cleaned,job_type_cleaned,min_exp,max_exp
0,"team skills, communication skills, analytical ...",Senior Exploit and Vulnerability Researcher,Exp: Minimum 5 years;Good understanding of IOC...,Delhi NCR(Vikas Puri),NaN,5-7 yrs,6to10,team skills communication skills analytical sk...,senior exploit and vulnerability researcher,exp minimum years good understanding of ioc ru...,delhi ncr vikas puri,missingjobtype,5,7
1,"ppc, logistics, inventory management, supply c...",Head SCM,He should have handled a team of atleast 5-6 d...,Sonepat,NaN,10-17 yrs,10to15,ppc logistics inventory management supply chai...,head scm,he should have handled a team of atleast direc...,sonepat,missingjobtype,10,17
2,"HR Analytics, Employee Engagement, Training, S...",Deputy Manager - Talent Management & Leadershi...,Must be an effective communicator (written & s...,Delhi NCR,Analytics,5-9 yrs,15to25,hr analytics employee engagement training succ...,deputy manager talent management leadership de...,must be an effective communicator written spok...,delhi ncr,analytics,5,9
3,"SQL, Javascript, Automation, Python, Ruby, Ana...",Associate Manager Data Engineering,7 - 10 years of overall experience in data e...,Bengaluru,Analytics,7-10 yrs,10to15,sql javascript automation python ruby analytic...,associate manager data engineering,years of overall experience in data engineeri...,bengaluru,analytics,7,10
4,"accounting, finance, cash flow, financial plan...",TS- GSA- Senior Analyst,Chartered Accountancy degree or MBA in Finance...,Gurgaon,NaN,1-3 yrs,3to6,accounting finance cash flow financial plannin...,ts gsa senior analyst,chartered accountancy degree or mba in finance...,gurgaon,missingjobtype,1,3


In [12]:
df_train['merged'] = (df_train['desig_cleaned'] + ' ' + df_train['job_desc_cleaned'] + ' ' + df_train['skills_cleaned']
                      + ' ' + df_train['job_type_cleaned'])

df_test['merged'] = (df_test['desig_cleaned'] + ' ' + df_test['job_desc_cleaned'] + ' ' + df_test['skills_cleaned']
                     + ' ' + df_test['job_type_cleaned'])

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train['salary'] = le.fit_transform(df_train['salary'])

### train test split

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(
    df_train[['merged', 'loc_cleaned', 'min_exp', 'max_exp']], 
    df_train['salary'], test_size=0.20, 
    stratify=df_train['salary'], random_state=75)

In [15]:
print('No. of sample texts X_train: ', len(X_train))
print('No. of sample texts X_cv   : ', len(X_cv))

No. of sample texts X_train:  15840
No. of sample texts X_cv   :  3961


### build the model & predict on CV 

In [16]:
X_train_merged = X_train['merged']
X_train_loc = X_train['loc_cleaned']

X_cv_merged = X_cv['merged']
X_cv_loc = X_cv['loc_cleaned']

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [18]:
tf1 = TfidfVectorizer(min_df=3, token_pattern=r'\w{3,}', ngram_range=(1,3), max_df=0.9)
tf2 = TfidfVectorizer(min_df=2, token_pattern=r'\w{3,}')

X_train_merged = tf1.fit_transform(X_train_merged)
X_train_loc = tf2.fit_transform(X_train_loc)

X_cv_merged = tf1.transform(X_cv_merged)
X_cv_loc = tf2.transform(X_cv_loc)

In [19]:
from sklearn.preprocessing import StandardScaler

sc1 = StandardScaler()
X_train_MinExp = sc1.fit_transform(np.array(X_train['min_exp']).reshape(-1,1))
X_cv_MinExp = sc1.transform(np.array(X_cv['min_exp']).reshape(-1,1))


sc2 = StandardScaler()
X_train_MaxExp = sc2.fit_transform(np.array(X_train['max_exp']).reshape(-1,1))
X_cv_MaxExp = sc2.transform(np.array(X_cv['max_exp']).reshape(-1,1))


In [20]:
merged_train = hstack((X_train_merged, X_train_loc, X_train_MinExp, X_train_MaxExp))
merged_cv  = hstack((X_cv_merged, X_cv_loc, X_cv_MinExp, X_cv_MaxExp))

In [21]:
merged_train.shape, merged_cv.shape

((15840, 52318), (3961, 52318))

In [22]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [23]:
import lightgbm as lgb
train_data = lgb.Dataset(merged_train, label=y_train)
test_data = lgb.Dataset(merged_cv, label=y_cv)

In [24]:
param = {'objective': 'multiclass',
         'num_iterations': 80,
         'learning_rate': 0.04,  
         'num_leaves': 23,
         'max_depth': 7, 
         'min_data_in_leaf': 28, 
         'max_bin': 10, 
         'min_data_in_bin': 3,   
         'num_class': 6,
         'metric': 'multi_logloss'
         }

In [25]:
lgbm = lgb.train(params=param,
                 train_set=train_data,
                 num_boost_round=100,
                 valid_sets=[test_data])

y_pred_class = lgbm.predict(merged_cv)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.054711 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 29632
[LightGBM] [Info] Number of data points in the train set: 15840, number of used features: 3005
[LightGBM] [Info] Start training from score -1.808567
[LightGBM] [Info] Start training from score -1.481605
[LightGBM] [Info] Start training from score -1.568616
[LightGBM] [Info] Start training from score -2.531427
[LightGBM] [Info] Start training from score -1.947616
[LightGBM] [Info] Start training from score -1.724030
[1]	valid_0's multi_logloss: 1.70572
[2]	valid_0's multi_logloss: 1.67071
[3]	valid_0's multi_logloss: 1.63995
[4]	valid_0's multi_logloss: 1.61241
[5]	valid_0's multi_logloss: 1.58778
[6]	valid_0's multi_logloss: 1.56475
[7]	valid_0's multi_logloss: 1.54412
[8]	valid_0's multi_logloss: 1.52528
[9]	valid_0's multi_l

In [26]:
predictions = []
for x in y_pred_class:
    predictions.append(np.argmax(x))

print('accuracy:', accuracy_score(y_cv, predictions))

accuracy: 0.47942438778086344


#### Predict on test set

In [27]:
X_test_merged = df_test['merged']
X_test_loc = df_test['loc_cleaned']

In [28]:
X_test_merged = tf1.transform(X_test_merged)
X_test_loc = tf2.transform(X_test_loc)

In [29]:
X_test_MinExp = sc1.transform(np.array(df_test['min_exp']).reshape(-1,1))
X_test_MaxExp = sc2.transform(np.array(df_test['max_exp']).reshape(-1,1))

In [30]:
merged_test  = hstack((X_test_merged, X_test_loc, X_test_MinExp, X_test_MaxExp))

In [31]:
predictions = lgbm.predict(merged_test)

y_pred_class = []
for x in predictions:
    y_pred_class.append(np.argmax(x))

y_pred_class = le.inverse_transform(y_pred_class)

In [32]:
df_sub = pd.DataFrame(data=y_pred_class, columns=['salary'])

In [33]:
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
df_sub.to_excel(writer, index=False)
writer.save()